In [ ]:
import os
import cv2
import random
import math
import numpy as np
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET')

all_classes = os.listdir('/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET')
print (all_classes)  
len(all_classes)

In [ ]:
plt.figure(figsize=(20,20))
random_range=random.sample(range(len(all_classes)),22)
for counter,random_index in enumerate(random_range,1):
  selected_Class=all_classes[random_index]
  video_files_names_list=os.listdir(f'/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET/{selected_Class}')
  selected_video_file_name=random.choice(video_files_names_list)
  video_reader=cv2.VideoCapture(f'/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET/{selected_Class}/{selected_video_file_name}')
  _,bgr_frame=video_reader.read()
  rgb_frame=cv2.cvtColor(bgr_frame,cv2.COLOR_BGR2RGB)
  cv2.putText(rgb_frame,selected_Class,(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
  plt.subplot(11,2,counter);plt.imshow(rgb_frame);plt.axis('off')

In [ ]:
IMAGE_HEIGHT,IMAGE_WIDTH=128,128
SEQUENCE_LENGTH=10
DATASET='/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET'
CLASSES_LIST=all_classes

In [ ]:
def frames_extraction(video_path):
  frames_list=[]
  video_reader=cv2.VideoCapture(video_path)
  video_frames_count=int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frames_window=max(int(video_frames_count/SEQUENCE_LENGTH),1)
  for frame_counter in range(SEQUENCE_LENGTH):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES,frame_counter*skip_frames_window)
    success,frame=video_reader.read()
    if not success:
      break
    resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
    normalized_frame=resized_frame/255
    frames_list.append(normalized_frame)
  video_reader.release()
  return frames_list

In [ ]:
def create_dataset():
  features=[]
  labels=[]
  video_files_path=[]
  for class_index,class_name in enumerate(CLASSES_LIST):
    print(f'Extracting the data of class:  {class_name}')
    files_list=os.listdir(os.path.join(DATASET,class_name))
    for file_name in files_list:
      video_file_path=os.path.join(DATASET,class_name,file_name)
      frames=frames_extraction(video_file_path)
      if len(frames)==SEQUENCE_LENGTH:
        features.append(frames)
        labels.append(class_index)
        video_files_path.append(video_file_path)
  features=np.asarray(features)
  labels=np.array(labels)
  return features,labels,video_files_path

In [ ]:
features,labels,video_files_path=create_dataset()

In [ ]:
from keras.utils import to_categorical
one_hot_encoded_labels=to_categorical(labels)

In [ ]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [ ]:
from sklearn.model_selection import train_test_split
features_train,features_test,labels_train,labels_test=train_test_split(features,one_hot_encoded_labels,test_size=0.2,shuffle=True,random_state=seed_constant)

In [ ]:
a=features_train.shape[0]

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, TimeDistributed, Dropout, Bidirectional, LSTM
from keras.applications import ResNet50

def create_resnet_bilstm_model():
    # Load pre-trained ResNet50 model without top layers
    resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    
    # Set ResNet layers as non-trainable (freeze them)
    for layer in resnet_model.layers:
        layer.trainable = False
    
    model = Sequential()
    
    # Add ResNet model as the first layer
    model.add(resnet_model)
    
    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(units=32, activation='relu', return_sequences=True)))
    model.add(Bidirectional(LSTM(units=64, activation='relu', return_sequences=True)))
    model.add(Flatten())
    model.add(Dense(len(CLASSES_LIST), activation='softmax'))
    
    model.summary()
    return model


In [ ]:
model1=create_resnet_bilstm_model()

In [ ]:
from keras.utils import plot_model
plot_model(model1,to_file='model1.png',show_shapes=True,show_layer_names=True)

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_call=EarlyStopping(monitor='val_loss',patience=10,mode='min',restore_best_weights=True)
model1.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
r=model1.fit(x=features_train,y=labels_train,epochs=50,batch_size=32,shuffle=True,validation_split=0.2,callbacks=[early_stopping_call])